In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# --- 데이터 로드 ---
# 전처리 완료된 데이터 삽입
df = pd.read_csv('adolescent_health_data.csv')

# --- 피처(X)와 타겟(y) 정의 ---
m_targets = [col for col in df.columns if col.startswith('M_')]
all_features = [col for col in df.columns if col not in m_targets]

X = df[all_features]
y = df[m_targets]

# --- 훈련/테스트 데이터 분할 ---
# 모든 모델이 이 데이터를 사용합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("훈련 데이터 크기:", X_train.shape, y_train.shape)
print("테스트 데이터 크기:", X_test.shape, y_test.shape)
print("타겟 변수:", m_targets)

훈련 데이터 크기: (43722, 173) (43722, 22)
테스트 데이터 크기: (10931, 173) (10931, 22)
타겟 변수: ['M_STR', 'M_SLP_EN', 'M_SLP_HR', 'M_SLP_MM', 'M_WK_HR', 'M_WK_MM', 'M_SLP_HR_K', 'M_SLP_MM_K', 'M_WK_HR_K', 'M_WK_MM_K', 'M_SAD', 'M_SUI_CON', 'M_SUI_PLN', 'M_SUI_ATT', 'M_LON', 'M_GAD_1', 'M_GAD_2', 'M_GAD_3', 'M_GAD_4', 'M_GAD_5', 'M_GAD_6', 'M_GAD_7']


### Baseline 1 - Linear Regression
(regularizaiton 적용?)

In [ ]:
# 1. 모델 생성 및 학습
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# 2. 예측
y_pred_lr = model_lr.predict(X_test)

# 3. 평가
# 평균 제곱 오차(Mean Squared Error, MSE)는 작을수록 좋습니다.
# R-squared(결정 계수)는 1에 가까울수록 좋습니다.
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("--- 선형 회귀 모델 평가 ---")
print(f"MSE: {mse_lr:.4f}")
print(f"R-squared: {r2_lr:.4f}")

### Baseline 2 - Random Forest

In [ ]:
# 1. 모델 생성 및 학습
# n_jobs=-1: 컴퓨터의 모든 CPU 코어를 사용하여 학습 속도를 높입니다.
model_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf.fit(X_train, y_train)

# 2. 예측
y_pred_rf = model_rf.predict(X_test)

# 3. 평가
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("--- 랜덤 포레스트 모델 평가 ---")
print(f"MSE: {mse_rf:.4f}")
print(f"R-squared: {r2_rf:.4f}")

# (참고) 만약 M_ 타겟이 확실한 분류(Classification) 문제라면
# from sklearn.ensemble import RandomForestClassifier 를 사용할 수도 있습니다.

### Baseline 3 - Basic ANN

In [ ]:
# 1. 모델 정의 (Keras Functional API 사용)
# 입력층: 전체 특성 개수만큼의 shape를 가짐
input_layer = Input(shape=(X_train.shape[1],), name='input_layer')

# 은닉층: 16개의 노드를 가진 Dense 레이어 1개
hidden_layer = Dense(16, activation='relu', name='hidden_layer')(input_layer)

# 출력층: 타겟 변수의 개수만큼 노드를 가짐
# 타겟이 0~1 사이 값이면 sigmoid, 일반적인 연속 값이면 linear가 적합합니다.
output_layer = Dense(len(m_targets), activation='sigmoid', name='output_layer')(hidden_layer)

model_ann = Model(inputs=input_layer, outputs=output_layer)

# 2. 모델 컴파일
# 손실 함수로 MSE를 사용하여 다른 모델들과 평가 지표를 통일합니다.
model_ann.compile(optimizer='adam', loss='mean_squared_error')

model_ann.summary()

# 3. 모델 학습
history = model_ann.fit(
    X_train,
    y_train,
    validation_split=0.2, # 훈련 데이터 중 일부를 검증용으로 사용
    epochs=20,
    batch_size=32,
    verbose=0 # 학습 과정 로그 생략
)

print("\n✅ 기본 ANN 모델 학습 완료!")

# 4. 예측 및 평가
y_pred_ann = model_ann.predict(X_test)
mse_ann = mean_squared_error(y_test, y_pred_ann)
r2_ann = r2_score(y_test, y_pred_ann)

print("\n--- 기본 ANN 모델 평가 ---")
print(f"MSE: {mse_ann:.4f}")
print(f"R-squared: {r2_ann:.4f}")